In [ ]:
import pandas as pd
import pymorphy2

In [ ]:
import psycopg2
conn = psycopg2.connect("host= dbname= user= port= password=")
conn.autocommit = True
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM records WHERE customer_id=1")
list_from_db = cur.fetchall()

In [ ]:
cur.execute("SELECT object FROM customer_objects WHERE customer_id=1")
list2_from_db = cur.fetchall()

In [ ]:
df = pd.DataFrame(list_from_db, columns=["id", "customer_id", "source", "category", "source_id", "additional_data", "text", "sentiment", "processed", "objects", "source_created_at", "created_at", "updated_at", "coverage", "coverage_data", "author_processed"])

In [ ]:
white_list=[]
for elem in list2_from_db:
    white_list.append(elem[0])

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def is_name(x):
    w = morph.parse(x)[0]
    if 'Name' in w.tag:
        return True
    else:
        return False

In [ ]:
def change_last_name(x):
    x1 = x.split(' ')
    if len(x1) == 2 and x.find('.') == -1 and is_name(x1[1]) == True:
        x2 = x1[1] + ' ' + x1[0]
        return x2
    else:
        return x

In [ ]:
def clean_objects(x):
    clean_objects=[]
    for elem in x:
        elem=change_last_name(elem)
        if elem in white_list:
            clean_objects.append(elem)
    return clean_objects

In [ ]:
df['objects'] = df['objects'].map(clean_objects)

In [ ]:
for row in df.iterrows():
    x = row[1]
    x['objects'] = '{' + ','.join(x['objects']) + '}'
    cur.execute("UPDATE records SET objects='{0}' WHERE id={1}".format(x['objects'],x['id']))

In [ ]:
cur.close()
conn.close()